In [ ]:
pip install transformers --quiet

In [ ]:
pip install -U sentence-transformers --quiet

In [ ]:
%%capture
!pip install git+https://github.com/MaartenGr/BERTopic.git@master

!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64

In [ ]:
import pandas as pd
import numpy as np
import os

from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_final = pd.read_csv('/content/drive/MyDrive/vkr/final_books.csv')
df_final.head()

,mark,publication_date,file_text,author,title,год попадания,автор/авторы,книга,цикл,аннотация,...,время,Ссылка на выпуск обзора.1,"год попадания, из исходной редакции, без ""округления""",Unnamed: 22,Unnamed: 23,Unnamed: 24,Служебные отметки,LC,link_domain,num_pers
0,3.6*84,-,Лернер_Марик_Совсем_не_прогрессор.txt,Лернер Марик,Совсем не прогрессор,2000,Лернер Марик,Совсем не прогрессор,NaN,Нормальный человек не стремится непременно все...,...,NaN,#Н/Д,2000.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,398
1,1.5*31,-,Нилан_Ник_Белый_архив.txt,Нилан Ник,Белый архив,1990,Нилан Ник,Белый архив,NaN,"Он скатился на дно, залез в долги, ввязался в ...",...,NaN,#Н/Д,1990.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,178
2,4.3*3,-,Малов_Владимир_Зачет_по_натуральной_истории.txt,Малов Владимир,Зачет по натуральной истории,1989,Малов Владимир,Зачет по натуральной истории,NaN,Урок ботаники в обычном классе обычной московс...,...,NaN,https://zen.yandex.ru/media/backinussr/novosti...,NaN,NaN,NaN,NaN,+++++,NaN,flibusta.is,93
3,1.4*7,-,Соловьев_Роман_Сквозь_время.txt,Соловьев Роман,Сквозь время,1989,Соловьев Роман,Сквозь время,NaN,"Мог ли подумать Сергей Павлович Кузнецов, когд...",...,NaN,https://zen.yandex.ru/media/backinussr/novosti...,NaN,NaN,NaN,NaN,+++++,NaN,flibusta.is,207
4,1.8*21,-,Рагорин_Алексей_Ракетчики.txt,Рагорин Алексей,Ракетчики,1988,Рагорин Алексей,Ракетчики,NaN,"Колониализм, общество разврата и потребления д...",...,NaN,#Н/Д,1988.0,NaN,NaN,NaN,+++++,NaN,flibusta.is,1612


In [ ]:
folder_books = '/content/drive/MyDrive/vkr/books'
books = []

for i, row in tqdm(df_final.iterrows()):
    file_text = row['file_text']
    with open(os.path.join(folder_books, file_text), 'r') as f:
        book = f.readlines()

    books.extend(book)

629it [00:14, 42.45it/s]


In [ ]:
len(books)

1372819

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = SentenceTransformer('cointegrated/rubert-tiny2')
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
embeddings = model.encode(books,
                          convert_to_tensor=True,
                          device=device,
                          show_progress_bar=True
                         )

KeyboardInterrupt: 

In [ ]:
with open('/content/drive/MyDrive/vkr/embeddings.npy', 'wb') as f:
    np.save(f, embeddings.cpu())

In [ ]:
embeddings = np.load('/content/drive/MyDrive/vkr/embeddings.npy')

In [ ]:
embeddings.shape

(1372819, 312)

## Обучаем BERTopic

In [ ]:
import collections
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Extract vocab to be used in BERTopic
vocab = collections.Counter()
tokenizer = CountVectorizer().build_tokenizer()
for paragraph in tqdm(books):
  vocab.update(tokenizer(paragraph))

vocab = [word for word, frequency in vocab.items() if frequency >= 10]

len(vocab)

100%|██████████| 1372819/1372819 [00:35<00:00, 38228.80it/s]


210933

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords = list(stopwords.words('russian'))

In [ ]:
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
from bertopic import BERTopic

In [ ]:
umap_model = UMAP(n_components=5, n_neighbors=50, random_state=42, metric="cosine", verbose=True)
hdbscan_model = HDBSCAN(min_samples=20, gen_min_span_tree=True, prediction_data=False, min_cluster_size=50, verbose=True)
vectorizer_model = CountVectorizer(vocabulary=vocab, stop_words=stopwords)

topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    verbose=True
).fit(books, embeddings=embeddings)

[D] [12:07:43.479726] /__w/cuml/cuml/cpp/src/umap/runner.cuh:107 n_neighbors=50
[D] [12:07:43.483181] /__w/cuml/cuml/cpp/src/umap/runner.cuh:129 Calling knn graph run
[D] [12:15:01.306233] /__w/cuml/cuml/cpp/src/umap/runner.cuh:135 Done. Calling fuzzy simplicial set
[D] [12:15:01.485153] /__w/cuml/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:317 Smooth kNN Distances
[D] [12:15:01.488733] /__w/cuml/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:319 sigmas = [ 0.0088142, 0.0199966, 0.00573823, 0.0104005, 0.0832404, 0.00630757, 0.078464, 0.101062, 0.0080345, 0.0842993, 0.121967, 0.0148803, 0.0234987, 0.0930701, 0.0128164, 4.46434e-05, 0.0800153, 0.0733395, 0.0352347, 0.0368087, 0.00992842, 4.46434e-05, 0.0138478, 0.0203939, 0.0157751 ]

[D] [12:15:01.488871] /__w/cuml/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:321 rhos = [ 0.157773, 0.184711, 0.119583, 0.15459, 5.96046e-08, 0.167808, 1.19209e-07, 1.78814e-07, 0.149033, 5.96046e-08, 5.96046e-08, 0.178592, 0.17678, 2.38419e-07, 0.211581, 0.002